In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import json
import os

In [3]:
result = requests.get('https://understat.com/')
src = result.content
soup = BeautifulSoup(src, 'html.parser')
Misa = soup.find_all("a")
League_links = []
for L in Misa:
    League_links.append(L.attrs['href'])
League_links_2 = []
for link in League_links:
    League_links_2.append('https://understat.com/' + link)
League_links_2 = League_links_2[:6]
years = ['/2014', '/2015', '/2016', '/2017', '/2018', '/2019', '/2020']
League_year = []
for link in League_links_2:
    for year in years:
        League_year.append(link + year)

In [3]:
OG_cols = ['Home/Away',
 'xG',
 'xGA',
 'non-penalty xG',
 'non-penalty xG Against',
 'Deep',
 'Deep Allowed',
 'scored',
 'conceded',
 'Expected Points',
 'Result',
 'date',
 'Wins',
 'Draws',
 'Loses',
 'Points',
 'Non-penalty xG difference',
 'ppda attack',
 'ppda defence',
 'ppda allowed att',
 'ppda allowed def',
 'Wins cum',
 'Loses cum',
 'Draws cum',
 'Points cum',
 'npxG difference cum',
 'xG cum',
 'xGA cum',
 'npxG cum',
 'npxGA cum',
 'scored cum',
 'conceded cum',
 'Expected Points cum',
 'Opponent',
 'Prob Win',
 'Prob Draw',
 'Prob Lose',
 'Wins in last 5',
 'Loses in last 5',
 'Draws in last 5',
 'Points in last 5',
 'npxG difference last 5',
 'xG last 5',
 'xGA last 5',
 'npxG last 5',
 'npxGA last 5',
 'scored last 5',
 'conceded last 5',
 'Expected Points last 5']
Opp_cols = []
for col in OG_cols:
    Opp_cols.append('Opp ' + col)

In [11]:
for link in League_links_2[:1]:
    s = link.rfind('/')
    league = link[s+1:]
    for year in years:
        result = requests.get(link + year)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        Misa = soup.find_all("script")
        Game_Table_Player_data = []
        for bruh in Misa[1:4]:
            bruh = str(bruh)
            s_1 = bruh.find('\'')
            s_2 = bruh.rfind('\'')
            bruh = bruh[s_1:s_2 + 1]
            Game_Table_Player_data.append(bruh)
        Game_ID = []
        is_result = []
        Home = []
        Away = []
        date_time = []
        Home_Goals = []
        Away_Goals = []
        xG_Home = []
        xG_Away = []
        Prob_Home_Win = []
        Prob_Draw = []
        Prob_Away_Win = []
        Game_data = json.loads(Game_Table_Player_data[0].encode('ascii').decode('unicode-escape')[1:-1])
        for Game in Game_data:
            Game['h'] = Game['h']['title']
            Game['a'] = Game['a']['title']
            Game['Home Goals'] = Game['goals']['h']
            Game['Away Goals'] = Game['goals']['a']
            Game['xG Home'] = Game['xG']['h']
            Game['xG Away'] = Game['xG']['a']
            if ('forecast' in list(Game.keys())):
                Game['Probability Home Win'] = Game['forecast']['w']
                Game['Probability Draw'] = Game['forecast']['d']
                Game['Probability Away Win'] = Game['forecast']['l']
                del Game['forecast']
                Prob_Home_Win.append(list(Game.values())[11])
                Prob_Draw.append(list(Game.values())[12])
                Prob_Away_Win.append(list(Game.values())[13])
            else:
                Prob_Home_Win.append('N/A')
                Prob_Draw.append('N/A')
                Prob_Away_Win.append('N/A')
            del Game['goals']
            del Game['xG']
            Game_ID.append(list(Game.values())[0])
            is_result.append(list(Game.values())[1])
            Home.append(list(Game.values())[2])
            Away.append(list(Game.values())[3])
            date_time.append(list(Game.values())[4])
            Home_Goals.append(list(Game.values())[5])
            Away_Goals.append(list(Game.values())[6])
            xG_Home.append(list(Game.values())[7])
            xG_Away.append(list(Game.values())[8])
        Game_Data = pd.DataFrame({'id': Game_ID, 'Is Result?': is_result, 'Home Team': Home, 
                                              'Away Team': Away, 'Home Goals': Home_Goals, 'Away Goals': Away_Goals,
                                          'xG Home' : xG_Home, 'xG Away': xG_Away, 'Probability Home Win': Prob_Home_Win,
                                             'Probability Draw': Prob_Draw, 'Probability Away Win': Prob_Away_Win})
        #Fixing up the dictionary given by the parsing
        Table_data = json.loads(Game_Table_Player_data[1].encode('ascii').decode('unicode-escape')[1:-1])
        for key in list(Table_data.keys()):
            Table_data[Table_data[key]['title']] = Table_data[key]
            del Table_data[key]
        for team in list(Table_data.keys()):
            Table_data[team] = Table_data[team]['history']
        #initial dataset
        for team in list(Table_data.keys()):
            h_a = []
            xG = []
            xGA = []
            npxG = []
            npxGA = []
            deep = []
            deep_allowed = []
            scored = []
            missed = []
            xpts = []
            result = []
            date = []
            wins = []
            draws = []
            loses = []
            pts = []
            npxGD = []
            ppda_att = []
            ppda_def = []
            ppda_allowed_att = []
            ppda_allowed_def = []
            for Game in Table_data[team]:
                Game['ppda att'] = Game['ppda']['att']
                Game['ppda def'] = Game['ppda']['def']
                Game['ppda allowed att'] = Game['ppda_allowed']['att']
                Game['ppda allowed def'] = Game['ppda_allowed']['def']
                del Game['ppda']
                del Game['ppda_allowed']
                h_a.append(list(Game.values())[0])
                xG.append(list(Game.values())[1])
                xGA.append(list(Game.values())[2])
                npxG.append(list(Game.values())[3])
                npxGA.append(list(Game.values())[4])
                deep.append(list(Game.values())[5])
                deep_allowed.append(list(Game.values())[6])
                scored.append(list(Game.values())[7])
                missed.append(list(Game.values())[8])
                xpts.append(list(Game.values())[9])
                result.append(list(Game.values())[10])
                date.append(list(Game.values())[11])
                wins.append(list(Game.values())[12])
                draws.append(list(Game.values())[13])
                loses.append(list(Game.values())[14])
                pts.append(list(Game.values())[15])
                npxGD.append(list(Game.values())[16])
                ppda_att.append(list(Game.values())[17])
                ppda_def.append(list(Game.values())[18])
                ppda_allowed_att.append(list(Game.values())[19])
                ppda_allowed_def.append(list(Game.values())[20])
            df = pd.DataFrame({'Home/Away': h_a, 'xG': xG, 'xGA': xGA, 'non-penalty xG': npxG, 
                                             'non-penalty xG Against': npxGA, 'Deep': deep, 'Deep Allowed' : deep_allowed, 
                                             'scored': scored, 'conceded': missed, 'Expected Points': xpts, 'Result': result,
                                         'date': date, 'Wins': wins, 'Draws': draws, 'Loses': loses, 'Points': pts,
                                            'Non-penalty xG difference': npxGD, 'ppda attack': ppda_att, 
                                             'ppda defence': ppda_def, 'ppda allowed att': ppda_allowed_att,
                                            'ppda allowed def': ppda_allowed_def})
            if (link + year == 'https://understat.com/league/Ligue_1/2019'):
                #Define lists
                one_to_ten = []
                for i in range(0,10):
                    one_to_ten.append(i)
                one_to_eleven = []
                for i in range(0,11):
                    one_to_eleven.append(i)
                #Now we actually fix up the Ligue 1 2019 issues
                if len(list(df['xG'])) == 28:
                    df_cancelled = pd.DataFrame(index = one_to_ten, columns=df.columns)
                    df_cancelled = df_cancelled.fillna(0)
                    df = pd.concat([df, df_cancelled], ignore_index=True)
                elif len(list(df['xG'])) == 27:
                    df_cancelled = pd.DataFrame(index = one_to_eleven, columns=df.columns)
                    df_cancelled = df_cancelled.fillna(0)
                    df = pd.concat([df, df_cancelled], ignore_index=True)
            # Cumulative statistics
            df['Wins cum'] = df['Wins'].cumsum()
            df['Loses cum'] = df['Loses'].cumsum()
            df['Draws cum'] = df['Draws'].cumsum()
            df['Points cum'] = df['Points'].cumsum()
            df['npxG difference cum'] = df['Non-penalty xG difference'].cumsum()
            df['xG cum'] = df['xG'].cumsum()
            df['xGA cum'] = df['xGA'].cumsum()
            df['npxG cum'] = df['non-penalty xG'].cumsum()
            df['npxGA cum'] = df['non-penalty xG Against'].cumsum()
            df['scored cum'] = df['scored'].cumsum()
            df['conceded cum'] = df['conceded'].cumsum()
            df['Expected Points cum'] = df['Expected Points'].cumsum()
            
            # Form statistics
            form_wins = []
            form_draws = []
            form_loses = []
            form_pts = []
            form_xg = []
            form_npxgdif = []
            form_xga = []
            form_npxg = []
            form_npxga = []
            form_scored = []
            form_conceded = []
            form_xpts = []
            for i in range(0, df.shape[0]):
                form_df = df[max(i-4, 0):i+1]
                form_wins.append(form_df['Wins'].cumsum()[i])
                form_draws.append(form_df['Draws'].cumsum()[i])
                form_loses.append(form_df['Loses'].cumsum()[i])
                form_pts.append(form_df['Points'].cumsum()[i])
                form_npxgdif.append(form_df['Non-penalty xG difference'].cumsum()[i])
                form_xg.append(form_df['xG'].cumsum()[i])
                form_xga.append(form_df['xGA'].cumsum()[i])
                form_npxg.append(form_df['non-penalty xG'].cumsum()[i])
                form_npxga.append(form_df['non-penalty xG Against'].cumsum()[i])
                form_scored.append(form_df['scored'].cumsum()[i])
                form_conceded.append(form_df['conceded'].cumsum()[i])
                form_xpts.append(form_df['Expected Points'].cumsum()[i])
            df['Wins in last 5'] = form_wins
            df['Loses in last 5'] = form_loses
            df['Draws in last 5'] = form_draws
            df['Points in last 5'] = form_pts
            df['npxG difference last 5'] = form_npxgdif
            df['xG last 5'] = form_xg
            df['xGA last 5'] = form_xga
            df['npxG last 5'] = form_npxg
            df['npxGA last 5'] = form_npxga
            df['scored last 5'] = form_scored
            df['conceded last 5'] = form_conceded
            df['Expected Points last 5'] = form_xpts
            
            #Getting key stats from Game data and putting them into the Team data
            rows_to_drop = []
            for i in range(0, Game_Data.shape[0]):
                if (Game_Data['Home Team'][i] != team and Game_Data['Away Team'][i] != team):
                    rows_to_drop.append(i)
            Team_Games = Game_Data.drop(rows_to_drop)
            Team_Opps = []
            Prob_Win = []
            Prob_Draw = []
            Prob_Lose = []
            for i in list(Team_Games.index):
                if Team_Games['Home Team'][i] != team:
                    Team_Opps.append(Team_Games['Home Team'][i])
                    Prob_Lose.append(Team_Games['Probability Home Win'][i])
                    Prob_Win.append(Team_Games['Probability Away Win'][i])
                    Prob_Draw.append(Team_Games['Probability Draw'][i])
                else:
                    Team_Opps.append(Team_Games['Away Team'][i])
                    Prob_Win.append(Team_Games['Probability Home Win'][i])
                    Prob_Lose.append(Team_Games['Probability Away Win'][i])
                    Prob_Draw.append(Team_Games['Probability Draw'][i])
            df['Opponent'] = Team_Opps
            df['Prob Win'] = Prob_Win
            df['Prob Draw'] = Prob_Draw
            df['Prob Lose'] = Prob_Lose
            df.to_csv('Understat_Team_Data/{}_teams'.format(league) + '{}'.format(year) + '/{}.csv'.format(team))
            
            
            #if not os.path.exists('Understat_Team_Data'):
              #  os.makedirs('Understat_Team_Data')
            #if not os.path.exists('Understat_Team_Data/{}_teams'.format(league) + '/{}'.format(year)):
             #   os.makedirs('Understat_Team_Data/{}_teams'.format(league) + '/{}'.format(year))
            #if not os.path.exists('Understat_Team_Data/{}_teams'.format(league) + '/{}'.format(year) + '/{}.csv'.format(team)):
             #   df.to_csv('Understat_Team_Data/{}_teams'.format(league) + '{}'.format(year) + '/{}.csv'.format(team))
            #else:
             #   print('Already have {}'.format(team) + '{}'.format(year))

In [14]:
#Set up so that the right teams are matched with the right league (and teams aren't mistaken for ID numbers...)
for link in League_links_2[:1]:
    s = link.rfind('/')
    league = link[s+1:]
    for year in years:
        result = requests.get(link + year)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        Misa = soup.find_all("script")
        Game_Table_Player_data = []
        for bruh in Misa[1:4]:
            bruh = str(bruh)
            s_1 = bruh.find('\'')
            s_2 = bruh.rfind('\'')
            bruh = bruh[s_1:s_2 + 1]
            Game_Table_Player_data.append(bruh)
        Table_data = json.loads(Game_Table_Player_data[1].encode('ascii').decode('unicode-escape')[1:-1])
        for key in list(Table_data.keys()):
            Table_data[Table_data[key]['title']] = Table_data[key]
            del Table_data[key]
        for team in list(Table_data.keys()):
            Table_data[team] = Table_data[team]['history']
        #Getting opponent data
        for team in list(Table_data.keys()):
            df = pd.read_csv('Understat_Team_Data/{}_teams'.format(league) + '{}'.format(year) + '/{}.csv'.format(team))
            #Make first row have 0s coz the teams haven't played yet
            for col in list(df.columns):
                if ('Unnamed' in col) or ('Opp ' in col):
                    del df[col]
            
            
            df_ = pd.DataFrame(index = [0], columns=df.columns)
            df_ = df_.fillna(0)
            # Making a dataframe which has the OPP data from the previous gameweek
            for i in range(1, len(list(df['Opponent']))):
                opp_df = pd.read_csv('Understat_Team_Data/{}_teams'.format(league) + '{}'.format(year) + '/{}.csv'.format(list(df['Opponent'])[i]))
                for col in list(opp_df.columns):
                    if ('Unnamed' in col) or ('Opp ' in col):
                        del opp_df[col]
                if (opp_df['Opponent'][i-1] == team):
                    jeff = opp_df.iloc[i-2].to_frame()
                    jeffT = jeff.T
                else:
                    jeff = opp_df.iloc[i-1].to_frame()
                    jeffT = jeff.T
                df_ = pd.concat([df_, jeffT], ignore_index=True)
            #Making new columns and adding them to the dataframe
            opp_dict = {OG_cols[i]: Opp_cols[i] for i in range(len(OG_cols))}
            df_2 = df_.rename(columns = opp_dict, inplace = False)
            df_3 = pd.concat([df, df_2], axis = 1)
            #df_3.to_csv('Understat_Team_Data/{}_teams'.format(league) + '{}'.format(year) + '/{}.csv'.format(team))

NameError: name 'OG_cols' is not defined

In [5]:
mults_of_thirty_eight = [i*38 for i in range(0, len(years))]

In [7]:
years_no_slash = []
for year in years:
    year = year[1:]
    years_no_slash.append(year)


In [8]:
print(years_no_slash)

['2014', '2015', '2016', '2017', '2018', '2019', '2020']


In [38]:
for link in League_links_2[:1]:
    s = link.rfind('/')
    league = link[s+1:]
    for year in years_no_slash:
        for team in list(Table_data.keys()):
            df = pd.read_csv('Understat_Team_Data/{}_teams'.format(league) + '/{}'.format(year) + '/{}.csv'.format(team))
            for i in range(1, len(list(df['Opponent']))):
                a = str(int(year)-1)
                if os.path.exists('Understat_Team_Data/{}_teams'.format(league) + '/' + str(a) + '/{}.csv'.format(list(df['Opponent'])[i])):
                    jeff = pd.read_csv('Understat_Team_Data/{}_teams'.format(league) + '/' + str(a) + '/{}.csv'.format(list(df['Opponent'])[i]))
                    df_ = jeff.iloc[-1].to_frame()
                    df_T = df_.T

   Unnamed: 0 Home/Away        xG       xGA non-penalty xG  \
37         37         a  0.399522  0.984196       0.399522   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.984196    3            2      0        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       7.301162        5.843734         7.301162                 4   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   6                     6.2923    Tottenham       0.2036   

   Opp Prob Draw Opp Prob Lose  
37        0.2652        0.5312  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG       xGA non-penalty xG  \
37         37         h  1.07504  0.118613        1.07504   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.118613    5            1      1        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       8.60

   Unnamed: 0 Home/Away       xG       xGA non-penalty xG  \
37         37         h  3.16614  0.569092        3.16614   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.569092   24            2      4        1  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37        7.89071         3.20038          7.12954                 7   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   1                     3.9993      Chelsea       0.3987   

   Opp Prob Draw Opp Prob Lose  
37        0.3248        0.2765  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG       xGA non-penalty xG  \
37         37         h  1.64216  0.416396        1.64216   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.416396    3            4      2        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       8.4279

   Unnamed: 0 Home/Away        xG      xGA non-penalty xG  \
37         37         a  0.118613  1.07504       0.118613   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                1.07504    1            5      0        1  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       8.048912        5.526925         6.429497                 4   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   8                     5.6976    Liverpool        0.619   

   Opp Prob Draw Opp Prob Lose  
37        0.2281        0.1529  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         h  2.52912  1.39272        1.76796   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                1.39272   17            1      3        1  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       6.572619

   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         h  2.17428  1.26755        2.17428   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                1.26755    3            1      5        1  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37      11.495843        4.029976         9.973503                 3   

   Opp conceded last 5 Opp Expected Points last 5      Opp Opponent  \
37                  10                     4.1444  Newcastle United   

   Opp Prob Win Opp Prob Draw Opp Prob Lose  
37        0.054          0.15         0.796  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG       xGA non-penalty xG  \
37         37         h  1.64216  0.416396        1.64216   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.416396    3            4      2        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37     

   Unnamed: 0 Home/Away        xG       xGA non-penalty xG  \
37         37         a  0.399522  0.984196       0.399522   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.984196    3            2      0        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       7.301162        5.843734         7.301162                 4   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   6                     6.2923    Tottenham       0.2036   

   Opp Prob Draw Opp Prob Lose  
37        0.2652        0.5312  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG       xGA non-penalty xG  \
37         37         h  1.07504  0.118613        1.07504   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37               0.118613    5            1      1        0  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       8.60

...

   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         a  1.02292  2.15707        1.02292   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.15707    8            4      1        6  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       6.820613        8.640194         6.820613                 6   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   4                     8.4551      Burnley       0.6337   

   Opp Prob Draw Opp Prob Lose  
37        0.2314        0.1349  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away        xG      xGA non-penalty xG  \
37         37         a  0.775853  2.44073       0.775853   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.44073    5           15      0        2  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       5.643315

   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         a  1.02292  2.15707        1.02292   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.15707    8            4      1        6  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       6.820613        8.640194         6.820613                 6   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   4                     8.4551      Burnley       0.6337   

   Opp Prob Draw Opp Prob Lose  
37        0.2314        0.1349  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away        xG      xGA non-penalty xG  \
37         37         h  0.654751  0.66113       0.654751   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                0.66113    5            6      0        1  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37        9.71052

   Unnamed: 0 Home/Away        xG      xGA non-penalty xG  \
37         37         a  0.775853  2.44073       0.775853   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.44073    5           15      0        2  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       5.643315        11.91775         5.643315                16   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   4                    10.3146      Swansea       0.6849   

   Opp Prob Draw Opp Prob Lose  
37        0.1878        0.1273  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         a  1.39272  2.52912        1.39272   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                1.76796    1           17      1        3  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       4.652024

   Unnamed: 0 Home/Away       xG      xGA non-penalty xG  \
37         37         a  1.02292  2.15707        1.02292   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.15707    8            4      1        6  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       6.820613        8.640194         6.820613                 6   

   Opp conceded last 5 Opp Expected Points last 5 Opp Opponent Opp Prob Win  \
37                   4                     8.4551      Burnley       0.6337   

   Opp Prob Draw Opp Prob Lose  
37        0.2314        0.1349  

[1 rows x 99 columns]
   Unnamed: 0 Home/Away        xG      xGA non-penalty xG  \
37         37         a  0.775853  2.44073       0.775853   

   non-penalty xG Against Deep Deep Allowed scored conceded  ...  \
37                2.44073    5           15      0        2  ...   

   Opp xGA last 5 Opp npxG last 5 Opp npxGA last 5 Opp scored last 5  \
37       5.643315

FileNotFoundError: [Errno 2] No such file or directory: 'Understat_Team_Data/EPL_teams/2015/Hull.csv'